Gcloud setting needed. See https://cloud.google.com/sdk/docs/install-sdk

List clusters

In [1]:
! gcloud container clusters list

NAME: lab-cluster
LOCATION: europe-west4-a
MASTER_VERSION: 1.27.3-gke.100
MASTER_IP: 34.91.224.227
MACHINE_TYPE: e2-medium
NODE_VERSION: 1.27.3-gke.100
NUM_NODES: 3
STATUS: RUNNING


Connect google cloud to local kubectl CLI. Replace your cluster name with lab-cluster

In [3]:
! gcloud container clusters get-credentials lab-cluster

Fetching cluster endpoint and auth data.
kubeconfig entry generated for lab-cluster.


Enable the Filestore CSI

In [4]:
! gcloud container clusters update lab-cluster --update-addons=GcpFilestoreCsiDriver=ENABLED

Updating lab-cluster...
done.
Updated [https://container.googleapis.com/v1/projects/cryptic-dahlia-413216/zones/europe-west4-a/clusters/lab-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/europe-west4-a/lab-cluster?project=cryptic-dahlia-413216


RBAC settings

In [5]:
! kubectl apply -f security\admin-role.yaml
! kubectl apply -f security\reader-role.yaml
! kubectl apply -f security\editor-role.yaml
   
! kubectl apply -f security\admin-serviceaccount.yaml
! kubectl apply -f security\rolebinding.yaml
! kubectl apply -f security\user-rolebinding.yaml

role.rbac.authorization.k8s.io/admin-role created
role.rbac.authorization.k8s.io/reader-role created
role.rbac.authorization.k8s.io/editor-role created
serviceaccount/admin-service-account unchanged
rolebinding.rbac.authorization.k8s.io/app-role-binding created
rolebinding.rbac.authorization.k8s.io/user created


Persistence storage claim

In [6]:
! kubectl apply -f gcloud\gcloud-pvc.yaml

persistentvolumeclaim/standard-pvc created


Network settings

In [8]:
! kubectl apply -f db\mongodb-service.yaml
! kubectl apply -f backend\backend-service.yaml
! kubectl apply -f backend\service-loadbalancer.yaml
! kubectl apply -f app\app-loadbalancer.yaml

service/mongodb-service created
service/backend-service unchanged
service/service-loadbalancer unchanged
service/webapp-loadbalancer unchanged


Get network IPs. If EXTERNAL-IP is pending, please wait for a while and re-execute

In [10]:
! kubectl get svc

NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
backend-service        ClusterIP      10.28.13.133   <none>          8000/TCP       49s
kubernetes             ClusterIP      10.28.0.1      <none>          443/TCP        23h
mongodb-service        ClusterIP      10.28.11.191   <none>          27017/TCP      41s
service-loadbalancer   LoadBalancer   10.28.4.244    35.234.165.21   80:30344/TCP   49s
webapp-loadbalancer    LoadBalancer   10.28.7.29     35.204.29.57    80:32146/TCP   48s


Put EXTERNAL-IP of service-loadbalancer to app\configmap-app.yaml
Put Cluster-IP of mongodb-service to backend\backend-config.yaml
```YAML
# app\configmap-app.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: server-address
data:
#  put the EXTERNAL-IP of serviceloadbalancer here
  SERVER_URL: "http://35.234.165.21"
```

```YAML
# backend\backend-service.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: backend-config
data:
# put the CLUSTER-IP of mongodb-service here
  MONGO_URL: "10.28.11.191"
```

Secret and configMap

In [11]:
! kubectl apply -f app\app-secret.yaml
! kubectl apply -f app\configmap-app.yaml
! kubectl apply -f backend\backend-secret.yaml
! kubectl apply -f backend\backend-config.yaml
! kubectl apply -f db\mongodb-secret.yaml
! kubectl apply -f db\mongodb-configmap.yaml

secret/webapp-registry-secret created
configmap/server-address created
secret/backend-secret created
configmap/backend-config created
secret/mongodb-secret created
configmap/mongodb-config created


Deploy database

In [13]:
! kubectl apply -f db\mongodb-deployment.yaml

deployment.apps/mongodb-deployment created


Set user/data to mongodb

In [15]:
! kubectl get pods

NAME                                  READY   STATUS    RESTARTS   AGE
mongodb-deployment-5b8bf7fbdf-xp8zw   1/1     Running   0          98s


When the pod is running, go inside the pod and create database, set user and pass. The settings will be stored within persistence volume, so after manipulate pods, they remains.

```BASE
kubectl exec -it mongodb-deployment-5b8bf7fbdf-xp8zw -- bash
```

```BASH
mongosh
```

```javascript
use admin
```
```javascript
db.createUser({
  user: "admin",
  pwd: "pass",
  roles: ["userAdminAnyDatabase", "readWriteAnyDatabase"]
})
```
```javascript
db.auth("admin", "pass")
```

Deploy frontend and backend.

In [19]:
! kubectl apply -f app\app-deployment.yaml
! kubectl apply -f backend\backend-deployment.yaml

deployment.apps/webapp-deployment created
service/webapp-service created
deployment.apps/backend-deployment created


Show services, get the EXTERNAL-IP of webapp-loadbalancer, open it on web.

In [20]:
! kubectl get svc

NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE
backend-service        ClusterIP      10.28.13.133   <none>          8000/TCP         27m
kubernetes             ClusterIP      10.28.0.1      <none>          443/TCP          24h
mongodb-service        ClusterIP      10.28.11.191   <none>          27017/TCP        27m
service-loadbalancer   LoadBalancer   10.28.4.244    35.234.165.21   80:30344/TCP     27m
webapp-loadbalancer    LoadBalancer   10.28.7.29     35.204.29.57    80:32146/TCP     27m
webapp-service         NodePort       10.28.2.226    <none>          5000:30100/TCP   4m47s
